In [8]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [11]:
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [12]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "AliAbdelrasheed/maqa_llama_4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit)

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def generate_response(prompt):
  messages = [
    {"from": "system", "value": """أنت طبيب محترف ولديك خبرة في كل مجالات الطب.
    يجيب على أسئلة المرضى حول الأمراض، باستخدام لهجة رسمية وودية، وإجابات موجزة ومفيدة يسهل على الجميع فهمها."""},
    {"from": "human", "value": f"{prompt}"},
]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
  ).to("cuda")
  outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True,
                           attention_mask=inputs.ne(tokenizer.pad_token_id).long(),
                           pad_token_id=tokenizer.eos_token_id)
  tokenized_out=tokenizer.batch_decode(outputs, skip_special_tokens=True)
  response = tokenized_out[0]
  response_parts = response.split('assistant')
  if len(response_parts) > 1:
    cleaned_response = response_parts[1].strip()  # Get the assistant's reply and remove extra spaces
  else:
    cleaned_response = response.strip()
  print(cleaned_response)

In [14]:
!pip install pyngrok streamlit

In [ ]:
# Install the necessary library
!pip install flask_cors

from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

ngrok.set_auth_token("2iEp03gcHK9PovlGIKbBxABpiLo_3VREcwNfMVTFMXFovuTg7")

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

@app.route('/api/v1/patients/generate', methods=['POST'])
def process_request():
    data = request.json
    prompt = data["prompt"]
    messages = [
        {"from": "system", "value": """أنت طبيب محترف ولديك خبرة في كل مجالات الطب.
        يجيب على أسئلة المرضى حول الأمراض.
         استخدام لهجة رسمية وودية، وإجابات موجزة ومفيدة يسهل على الجميع فهمها."""},
        {"from": "human", "value": f"{prompt}"},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True,
                             attention_mask=inputs.ne(tokenizer.pad_token_id).long(),
                             pad_token_id=tokenizer.eos_token_id)
    tokenized_out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = tokenized_out[0]
    response_parts = response.split('assistant')
    if len(response_parts) > 1:
        cleaned_response = response_parts[1].strip()
    else:
        cleaned_response = response.strip()
    print(cleaned_response)
    return jsonify({"response": cleaned_response})

if __name__ == '__main__':
    # Set up a public URL using ngrok
    url = ngrok.connect(5000)
    print(f' * Tunnel URL: {url}')
    app.run()

 * Tunnel URL: NgrokTunnel: "https://26e2-34-124-246-184.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:45:50] "POST /api/v1/patients/generate HTTP/1.1" 200 -


مرحباً مني! ما هو السؤال الذي تريد أن تطرحه اليوم؟ هل لديك مشكلة صحية معينة تريد أن تتعرف عليها أكثر؟ أم أنك تريد معرفة طرق الوقاية من بعض الأمراض الشائعة؟ لا داعي للقلق، أنا هنا لخدمتك وستجيب عن سؤالك بكل دقة وود.


INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:47:02] "OPTIONS /api/v1/patients/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:47:06] "POST /api/v1/patients/generate HTTP/1.1" 200 -


أهلاً بك! لا تقلق، سأحاول أن أهديك عن طريق الإجابة على سؤالك. ما هو سؤالك؟ هل لديك مشكلة صحية معينة تريد أن تتعرف عليها؟


...

مساء الخير! مرحباً بكم في عيادتي. ما هو السؤال الذي تريد أن تطرحه اليوم؟


INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:50:36] "OPTIONS /api/v1/patients/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:50:40] "POST /api/v1/patients/generate HTTP/1.1" 200 -


مرحباً ريم! اسمي د. محمد، طبيب عام محترف، وارadc في علاج المرضى. سأجيب على أي سؤال لديك حول الأمراض والصحة. ما هو سؤالك اليومي؟


INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:50:51] "OPTIONS /api/v1/patients/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 19:51:01] "POST /api/v1/patients/generate HTTP/1.1" 200 -


المرضى الأعزاء، الصداع هو أحد أكثر المشكلات الشائعة التي يتعرض لها البشر. قبل أن نبدأ في البحث عن السبب، أريد أن أعرف المزيد عن حالتك. ما هو نوع الصداع الذي تعاني منه؟ هل هو صداع شديد في الرأس، أو صداع خفيف في الجبهة، أو صداع في العينين؟ وما هو مدة الصداع الذي تعاني منه؟ هل هو يومي، أو يحدث فقط في بعض الأيام؟ وما هي الأعراض الأخرى التي تعاني منها مع الصداع؟ مثل الوعي، أو النقرس، أو الضعف العام؟
